In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], inplace=True)

# Determine the number of unique values in each column.
unique_count = application_df.nunique()

# Look at APPLICATION_TYPE value counts for binning
application_type_count = application_df["APPLICATION_TYPE"].value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(application_type_count[application_type_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Look at CLASSIFICATION value counts for binning
classification_count = application_df["CLASSIFICATION"].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")


In [3]:
# EXTRA CHANGES TO DATA
# Remove features
print(application_df["STATUS"].value_counts())

application_df.drop(columns=["STATUS"], inplace=True)

print(application_df["SPECIAL_CONSIDERATIONS"].value_counts())

application_df.drop(columns=["SPECIAL_CONSIDERATIONS"], inplace=True)

# Bin features
print(application_df["USE_CASE"].value_counts())

use_case_count = application_df["USE_CASE"].value_counts()

replace_application = list(use_case_count[use_case_count < 500].index)

for use_case in replace_application:
    application_df.USE_CASE = application_df.USE_CASE.replace(use_case,"Other")
    
print(application_df["USE_CASE"].value_counts())

print(application_df["INCOME_AMT"].value_counts())

income_amt_count = application_df["INCOME_AMT"].value_counts()

replace_application = list(income_amt_count[income_amt_count < 500].index)

for use_case in replace_application:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(use_case,"5M+")
    
print(application_df["INCOME_AMT"].value_counts())

1    34294
0        5
Name: STATUS, dtype: int64
N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64
Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64


In [4]:
print(application_df["AFFILIATION"].value_counts())
print(application_df["ORGANIZATION"].value_counts())
print(application_df["ASK_AMT"].value_counts())

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64
Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64
5000       25398
15583          3
6725           3
63981          3
10478          3
           ...  
75598          1
22336          1
44863          1
5928           1
6948863        1
Name: ASK_AMT, Length: 8747, dtype: int64


In [5]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)

# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Attempt 1

In [6]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 70
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="gelu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="gelu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
print(nn.summary())

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               3700      
_________________________________________________________________
dense_1 (Dense)              (None, 70)                7070      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 71        
Total params: 10,841
Trainable params: 10,841
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Train the model
checkpoint_path = "./AlphabetSoupCharity_Optimization.hdf5"

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

fit_model = nn.fit(X_train_scaled,y_train,epochs=300,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/300
804/804 [==============================] - 1s 670us/step - loss: 0.7404 - accuracy: 0.7011
Epoch 2/300
804/804 [==============================] - 1s 675us/step - loss: 0.6253 - accuracy: 0.7252
Epoch 3/300
804/804 [==============================] - 1s 673us/step - loss: 0.5856 - accuracy: 0.7241
Epoch 4/300
804/804 [==============================] - 1s 679us/step - loss: 0.5809 - accuracy: 0.7277
Epoch 5/300
804/804 [==============================] - 1s 676us/step - loss: 0.5718 - accuracy: 0.7319

Epoch 00005: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 6/300
804/804 [==============================] - 1s 684us/step - loss: 0.5697 - accuracy: 0.7258
Epoch 7/300
804/804 [==============================] - 1s 670us/step - loss: 0.5584 - accuracy: 0.7328
Epoch 8/300
804/804 [==============================] - 1s 693us/step - loss: 0.5616 - accuracy: 0.7316
Epoch 9/300
804/804 [==============================] - 1s 689us/step - loss: 0.5584 - accuracy: 0.7317
Epo

804/804 [==============================] - 1s 690us/step - loss: 0.5412 - accuracy: 0.7383
Epoch 70/300
804/804 [==============================] - 1s 683us/step - loss: 0.5423 - accuracy: 0.7388

Epoch 00070: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 71/300
804/804 [==============================] - 1s 693us/step - loss: 0.5404 - accuracy: 0.7404
Epoch 72/300
804/804 [==============================] - 1s 689us/step - loss: 0.5418 - accuracy: 0.7388
Epoch 73/300
804/804 [==============================] - 1s 669us/step - loss: 0.5440 - accuracy: 0.7379
Epoch 74/300
804/804 [==============================] - 1s 683us/step - loss: 0.5397 - accuracy: 0.7374
Epoch 75/300
804/804 [==============================] - 1s 695us/step - loss: 0.5411 - accuracy: 0.7374

Epoch 00075: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 76/300
804/804 [==============================] - 1s 696us/step - loss: 0.5416 - accuracy: 0.7381
Epoch 77/300
804/804 [===================

804/804 [==============================] - 1s 663us/step - loss: 0.5378 - accuracy: 0.7413
Epoch 139/300
804/804 [==============================] - 1s 666us/step - loss: 0.5405 - accuracy: 0.7398
Epoch 140/300
804/804 [==============================] - 1s 709us/step - loss: 0.5392 - accuracy: 0.7403

Epoch 00140: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 141/300
804/804 [==============================] - 1s 679us/step - loss: 0.5419 - accuracy: 0.7401
Epoch 142/300
804/804 [==============================] - 1s 708us/step - loss: 0.5453 - accuracy: 0.7390
Epoch 143/300
804/804 [==============================] - 1s 686us/step - loss: 0.5436 - accuracy: 0.7393
Epoch 144/300
804/804 [==============================] - 1s 703us/step - loss: 0.5426 - accuracy: 0.7387
Epoch 145/300
804/804 [==============================] - 1s 699us/step - loss: 0.5402 - accuracy: 0.7383

Epoch 00145: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 146/300
804/804 [===========

804/804 [==============================] - 1s 670us/step - loss: 0.6404 - accuracy: 0.7368
Epoch 207/300
804/804 [==============================] - 1s 673us/step - loss: 0.5824 - accuracy: 0.7387
Epoch 208/300
804/804 [==============================] - 1s 673us/step - loss: 0.5722 - accuracy: 0.7407
Epoch 209/300
804/804 [==============================] - 1s 671us/step - loss: 0.5709 - accuracy: 0.7404
Epoch 210/300
804/804 [==============================] - 1s 674us/step - loss: 0.5710 - accuracy: 0.7400

Epoch 00210: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 211/300
804/804 [==============================] - 1s 674us/step - loss: 0.5720 - accuracy: 0.7399
Epoch 212/300
804/804 [==============================] - 1s 661us/step - loss: 0.5635 - accuracy: 0.7392
Epoch 213/300
804/804 [==============================] - 1s 665us/step - loss: 0.5634 - accuracy: 0.7391
Epoch 214/300
804/804 [==============================] - 1s 665us/step - loss: 0.5591 - accuracy: 0.7400

804/804 [==============================] - 1s 666us/step - loss: 0.5557 - accuracy: 0.7414

Epoch 00275: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 276/300
804/804 [==============================] - 1s 671us/step - loss: 0.5540 - accuracy: 0.7411
Epoch 277/300
804/804 [==============================] - 1s 666us/step - loss: 0.6279 - accuracy: 0.7364
Epoch 278/300
804/804 [==============================] - 1s 668us/step - loss: 0.5502 - accuracy: 0.7408
Epoch 279/300
804/804 [==============================] - 1s 668us/step - loss: 0.5505 - accuracy: 0.7406
Epoch 280/300
804/804 [==============================] - 1s 674us/step - loss: 0.5502 - accuracy: 0.7414

Epoch 00280: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 281/300
804/804 [==============================] - 1s 669us/step - loss: 0.5517 - accuracy: 0.7413
Epoch 282/300
804/804 [==============================] - 1s 670us/step - loss: 0.5519 - accuracy: 0.7402
Epoch 283/300
804/804 [===========

# Attempt 2

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  30
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="gelu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
print(nn.summary())

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 30)                1110      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 31        
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Train the model
checkpoint_path = "./AlphabetSoupCharity_Optimization.hdf5"

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

fit_model = nn.fit(X_train_scaled,y_train,epochs=300,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/300
804/804 [==============================] - 1s 528us/step - loss: 0.5755 - accuracy: 0.7201
Epoch 2/300
804/804 [==============================] - 0s 517us/step - loss: 0.5581 - accuracy: 0.7278
Epoch 3/300
804/804 [==============================] - 0s 521us/step - loss: 0.5550 - accuracy: 0.7290
Epoch 4/300
804/804 [==============================] - 0s 526us/step - loss: 0.5532 - accuracy: 0.7292
Epoch 5/300
804/804 [==============================] - 0s 533us/step - loss: 0.5515 - accuracy: 0.7318

Epoch 00005: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 6/300
804/804 [==============================] - 0s 514us/step - loss: 0.5508 - accuracy: 0.7309
Epoch 7/300
804/804 [==============================] - 0s 516us/step - loss: 0.5500 - accuracy: 0.7323
Epoch 8/300
804/804 [==============================] - 0s 517us/step - loss: 0.5494 - accuracy: 0.7322
Epoch 9/300
804/804 [==============================] - 0s 518us/step - loss: 0.5488 - accuracy: 0.7332
Epo

804/804 [==============================] - 0s 509us/step - loss: 0.5406 - accuracy: 0.7375
Epoch 70/300
804/804 [==============================] - 0s 508us/step - loss: 0.5404 - accuracy: 0.7365

Epoch 00070: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 71/300
804/804 [==============================] - 0s 509us/step - loss: 0.5403 - accuracy: 0.7363
Epoch 72/300
804/804 [==============================] - 0s 508us/step - loss: 0.5396 - accuracy: 0.7374
Epoch 73/300
804/804 [==============================] - 0s 511us/step - loss: 0.5402 - accuracy: 0.7372
Epoch 74/300
804/804 [==============================] - 0s 506us/step - loss: 0.5400 - accuracy: 0.7381
Epoch 75/300
804/804 [==============================] - 0s 507us/step - loss: 0.5401 - accuracy: 0.7370

Epoch 00075: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 76/300
804/804 [==============================] - 0s 514us/step - loss: 0.5400 - accuracy: 0.7368
Epoch 77/300
804/804 [===================

804/804 [==============================] - 0s 527us/step - loss: 0.5385 - accuracy: 0.7381
Epoch 139/300
804/804 [==============================] - 0s 529us/step - loss: 0.5386 - accuracy: 0.7381
Epoch 140/300
804/804 [==============================] - 0s 511us/step - loss: 0.5382 - accuracy: 0.7393

Epoch 00140: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 141/300
804/804 [==============================] - 0s 516us/step - loss: 0.5381 - accuracy: 0.7387
Epoch 142/300
804/804 [==============================] - 0s 511us/step - loss: 0.5385 - accuracy: 0.7374
Epoch 143/300
804/804 [==============================] - 0s 512us/step - loss: 0.5385 - accuracy: 0.7384
Epoch 144/300
804/804 [==============================] - 0s 517us/step - loss: 0.5384 - accuracy: 0.7388
Epoch 145/300
804/804 [==============================] - 0s 526us/step - loss: 0.5384 - accuracy: 0.7388

Epoch 00145: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 146/300
804/804 [===========

804/804 [==============================] - 0s 511us/step - loss: 0.5373 - accuracy: 0.7388
Epoch 207/300
804/804 [==============================] - 0s 521us/step - loss: 0.5374 - accuracy: 0.7383
Epoch 208/300
804/804 [==============================] - 0s 512us/step - loss: 0.5372 - accuracy: 0.7387
Epoch 209/300
804/804 [==============================] - 0s 516us/step - loss: 0.5375 - accuracy: 0.7391
Epoch 210/300
804/804 [==============================] - 0s 536us/step - loss: 0.5373 - accuracy: 0.7392

Epoch 00210: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 211/300
804/804 [==============================] - 0s 516us/step - loss: 0.5376 - accuracy: 0.7387
Epoch 212/300
804/804 [==============================] - 0s 517us/step - loss: 0.5371 - accuracy: 0.7404
Epoch 213/300
804/804 [==============================] - 0s 514us/step - loss: 0.5370 - accuracy: 0.7397
Epoch 214/300
804/804 [==============================] - 0s 538us/step - loss: 0.5373 - accuracy: 0.7392

804/804 [==============================] - 0s 506us/step - loss: 0.5367 - accuracy: 0.7396

Epoch 00275: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 276/300
804/804 [==============================] - 0s 519us/step - loss: 0.5371 - accuracy: 0.7391
Epoch 277/300
804/804 [==============================] - 0s 511us/step - loss: 0.5367 - accuracy: 0.7404
Epoch 278/300
804/804 [==============================] - 0s 511us/step - loss: 0.5367 - accuracy: 0.7387
Epoch 279/300
804/804 [==============================] - 0s 514us/step - loss: 0.5366 - accuracy: 0.7390
Epoch 280/300
804/804 [==============================] - 0s 511us/step - loss: 0.5366 - accuracy: 0.7390

Epoch 00280: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 281/300
804/804 [==============================] - 0s 509us/step - loss: 0.5369 - accuracy: 0.7386
Epoch 282/300
804/804 [==============================] - 0s 513us/step - loss: 0.5363 - accuracy: 0.7402
Epoch 283/300
804/804 [===========

# Attempt 3

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64
hidden_nodes_layer4 = 32
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="gelu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="gelu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="gelu"))

# Forth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
print(nn.summary())

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               9472      
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 54,849
Trainable params: 54,849
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Train the model
checkpoint_path = "./AlphabetSoupCharity_Optimization.hdf5"

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

fit_model = nn.fit(X_train_scaled,y_train,epochs=300,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/300
804/804 [==============================] - 1s 943us/step - loss: 0.5652 - accuracy: 0.7262
Epoch 2/300
804/804 [==============================] - 1s 944us/step - loss: 0.5548 - accuracy: 0.7304
Epoch 3/300
804/804 [==============================] - 1s 940us/step - loss: 0.5513 - accuracy: 0.7323
Epoch 4/300
804/804 [==============================] - 1s 940us/step - loss: 0.5499 - accuracy: 0.7304
Epoch 5/300
804/804 [==============================] - 1s 937us/step - loss: 0.5487 - accuracy: 0.7330

Epoch 00005: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 6/300
804/804 [==============================] - 1s 952us/step - loss: 0.5471 - accuracy: 0.7339
Epoch 7/300
804/804 [==============================] - 1s 947us/step - loss: 0.5466 - accuracy: 0.7348
Epoch 8/300
804/804 [==============================] - 1s 945us/step - loss: 0.5460 - accuracy: 0.7359
Epoch 9/300
804/804 [==============================] - 1s 948us/step - loss: 0.5448 - accuracy: 0.73440s -

804/804 [==============================] - 1s 968us/step - loss: 0.5318 - accuracy: 0.7418
Epoch 70/300
804/804 [==============================] - 1s 988us/step - loss: 0.5320 - accuracy: 0.7408

Epoch 00070: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 71/300
804/804 [==============================] - 1s 972us/step - loss: 0.5315 - accuracy: 0.7423
Epoch 72/300
804/804 [==============================] - 1s 955us/step - loss: 0.5318 - accuracy: 0.7420
Epoch 73/300
804/804 [==============================] - 1s 959us/step - loss: 0.5321 - accuracy: 0.7411
Epoch 74/300
804/804 [==============================] - 1s 984us/step - loss: 0.5329 - accuracy: 0.7414
Epoch 75/300
804/804 [==============================] - 1s 949us/step - loss: 0.5326 - accuracy: 0.7414

Epoch 00075: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 76/300
804/804 [==============================] - 1s 949us/step - loss: 0.5319 - accuracy: 0.7420
Epoch 77/300
804/804 [===================

804/804 [==============================] - 1s 957us/step - loss: 0.5307 - accuracy: 0.7423
Epoch 137/300
804/804 [==============================] - 1s 985us/step - loss: 0.5309 - accuracy: 0.7425
Epoch 138/300
804/804 [==============================] - 1s 950us/step - loss: 0.5310 - accuracy: 0.7416
Epoch 139/300
804/804 [==============================] - 1s 944us/step - loss: 0.5308 - accuracy: 0.7423
Epoch 140/300
804/804 [==============================] - 1s 943us/step - loss: 0.5309 - accuracy: 0.7411

Epoch 00140: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 141/300
804/804 [==============================] - 1s 943us/step - loss: 0.5314 - accuracy: 0.7425
Epoch 142/300
804/804 [==============================] - 1s 960us/step - loss: 0.5319 - accuracy: 0.7409
Epoch 143/300
804/804 [==============================] - 1s 945us/step - loss: 0.5318 - accuracy: 0.7416
Epoch 144/300
804/804 [==============================] - 1s 945us/step - loss: 0.5306 - accuracy: 0.7418

804/804 [==============================] - 1s 972us/step - loss: 0.5298 - accuracy: 0.7418
Epoch 205/300
804/804 [==============================] - 1s 949us/step - loss: 0.5304 - accuracy: 0.7424

Epoch 00205: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 206/300
804/804 [==============================] - 1s 952us/step - loss: 0.5300 - accuracy: 0.7425
Epoch 207/300
804/804 [==============================] - 1s 992us/step - loss: 0.5293 - accuracy: 0.7420
Epoch 208/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5299 - accuracy: 0.7416
Epoch 209/300
804/804 [==============================] - 1s 990us/step - loss: 0.5299 - accuracy: 0.7424
Epoch 210/300
804/804 [==============================] - 1s 942us/step - loss: 0.5296 - accuracy: 0.7424

Epoch 00210: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 211/300
804/804 [==============================] - 1s 960us/step - loss: 0.5302 - accuracy: 0.7424
Epoch 212/300
804/804 [=============

804/804 [==============================] - 1s 972us/step - loss: 0.5296 - accuracy: 0.7420
Epoch 272/300
804/804 [==============================] - 1s 957us/step - loss: 0.5294 - accuracy: 0.7418
Epoch 273/300
804/804 [==============================] - 1s 968us/step - loss: 0.5307 - accuracy: 0.7421
Epoch 274/300
804/804 [==============================] - 1s 973us/step - loss: 0.5307 - accuracy: 0.74200s - loss: 0.5301 - accuracy - ETA: 0s - loss: 0.5277 - 
Epoch 275/300
804/804 [==============================] - 1s 962us/step - loss: 0.5303 - accuracy: 0.7422

Epoch 00275: saving model to .\AlphabetSoupCharity_Optimization.hdf5
Epoch 276/300
804/804 [==============================] - 1s 945us/step - loss: 0.5295 - accuracy: 0.7424
Epoch 277/300
804/804 [==============================] - 1s 954us/step - loss: 0.5299 - accuracy: 0.7419
Epoch 278/300
804/804 [==============================] - 1s 954us/step - loss: 0.5295 - accuracy: 0.7420
Epoch 279/300
804/804 [=========================